# Анализ пространственных данных. Домашнее задание №2

Мягкий дедлайн: __4 ноября 2020 г. 23:59__

Жесткий дедлайн (со штрафом в _50%_ от количества набранных вами за ДЗ баллов): __5 ноября 2020 г. 08:59__

Визуализация "чего-либо" __без__ выполненного основного задания оценивается в __0 баллов__

ФИО: `Салынский Александр Викторович`

Группа: `DS-12`

## Задание №1. Горячая точка (алгоритм - 10 баллов, визуализация - 10 баллов).

Генерируйте рандомные точки на планете Земля до тех пор, пока не попадете на территорию ``Афганистана``

1. Вы можете использовать функции принадлжености точки полигону и расстояния от точки до полигона (в метрах)
2. Предложите не наивный алгоритм поиска (генерировать __напрямую__ точку из полигона границ Афганистана __запрещено__)

In [4]:
from collections import OrderedDict

from OSMPythonTools.nominatim import Nominatim
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
from OSMPythonTools.data import Data, ALL
from shapely.geometry import Point
import osmapi as osm
from random import uniform
from geopy.distance import distance, lonlat
import re
import geopandas as gpd
import folium
import pandas as pd
from countryinfo import CountryInfo


api = osm.OsmApi(username='geo-course', 
                 passwordfile='../config/api_key.txt')
nominatim = Nominatim()
overpass = Overpass()

In [5]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
polygon_afg = world.loc[world.iso_a3 == 'AFG'].geometry.values[0]

def find_min_dist(poly, point) -> float:
    min_dist = 45_000
    for coord in poly.exterior.coords:
        min_dist = min(min_dist, distance(coord, (point.x, point.y)))
    return min_dist

used_points = []

points = [{}, {}] 
points[0]['point'] = Point(-90, -180)
points[1]['point'] = Point(90, 180)

found = False
if polygon_afg.contains(points[0]['point']):
    found = True
    point = points[0]['point']
if polygon_afg.contains(points[1]['point']):
    found = True
    point = points[1]['point']

points[0]['dist'] = find_min_dist(polygon_afg, points[0]['point'])
points[1]['dist'] = find_min_dist(polygon_afg, points[1]['point'])

used_points.append(points[0]['point'])
used_points.append(points[1]['point'])

while not found:
    p_upd = 0
    if points[0]['dist'] < points[1]['dist']:
        p_upd = 1
        
    point_lat = Point((points[0]['point'].x + points[1]['point'].x) / 2, points[p_upd]['point'].y)
    point_lon = Point(points[p_upd]['point'].x, (points[0]['point'].y + points[1]['point'].y) / 2)
    dist_lon = find_min_dist(polygon_afg, point_lon)
    dist_lat = find_min_dist(polygon_afg, point_lat)
    
    points[p_upd]['point'] = point_lon
    points[p_upd]['dist'] = dist_lon
    if dist_lon > dist_lat:
        points[p_upd]['point'] = point_lat
        points[p_upd]['dist'] = dist_lat
        
    if polygon_afg.contains(points[p_upd]['point']):
        found = True
        point = points[p_upd]['point']
        
    used_points.append(points[p_upd]['point'])

In [6]:
area_id_af = nominatim.query('Afganistan').areaId()

query = overpassQueryBuilder(area=area_id_af, elementType='node', selector='"place"="country"', out='body')
center_node = overpass.query(query).nodes()[0]

print(f'Central coordinates for AFG: {center_node.lon(), center_node.lat()}')

Central coordinates for AFG: (66.2385139, 33.7680065)


In [7]:
m = folium.Map((center_node.lat(), center_node.lon()), 
               zoom_start=1, 
               tiles='Stamen Toner')

tooltip = 'Check step'

for i, point in enumerate(used_points):
    folium.Marker([point.y, point.x], 
                  popup=f'<i>Point received on step: {i}</i>', 
                  tooltip=tooltip,
                  icon=folium.Icon(color='red', icon='info-sign')).add_to(m)
m

Визуализируйте пошагово предложенный алгоритм при помощи ``Folium``

## Задание №2. Качество жизни (20 баллов).

Для измерения показателя качества жизни в точке, найденной в предыдущем задании, вам необходимо рассчитать следующую сумму расстояний (в метрах):

1. Расстояние от точки до 5 ближайших __*__ банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости
2. Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице
3. Расстояние от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части __**__

__*__ При поиске _N_ ближайших объектов обязательно использовать ``R-tree``

__**__ Южной частью страны является территория, находящаяся к югу от множества точек, равноудаленных от самой северной и самой южной точек страны

### 1. Расстояние от точки до 5 ближайших __*__ банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости

In [8]:
result = overpass.query('relation["admin_level"="2"][boundary=administrative]; out;')

pattern = re.compile(r' -|— ')
countries = []

for rel in result.relations():
    if not pattern.findall(rel.tag('name:en')) and not rel.id() % 10:
        countries.append(rel)

In [105]:
def count_residental_buildings(country) -> int:
    query = overpassQueryBuilder(area=country.id() + 3_600_000_000,
                                 elementType=['relation', 'way'],
                                 selector=[f'"building"="residential"',
                                           f'"building"="apartments"',
                                           f'"building"="house"'],
                                 out='count')
    return overpass.query(query, timeout=60).countElements()

dimensions = OrderedDict({
    'country': OrderedDict({
        country.tag('name:en'): country
        for country in countries
    })
})

data = Data(count_residental_buildings, dimensions)

df_res_buildings = data.getDataFrame().sort_values('value', ascending=False)
df_res_buildings.head()

,value
country,
Brazil,593024
Australia,492702
Portugal,194773
Norway,168952
Tanzania,59808


In [106]:
country_1 = df_res_buildings.iloc[0].name
country_1

'Brazil'

In [10]:
country_1_id = nominatim.query(country_1).areaId()
query = overpassQueryBuilder(area=country_1_id,
                             elementType='node',
                             selector=[f'"amenity"="atm"'],
                             out='body')
res = overpass.query(query, timeout=60)

In [11]:
print(f"Total number of ATM's in {country_1}: {res.countElements()}")

Total number of ATM's in Brazil: 1332


### 2. Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице

In [97]:
nominatim = Nominatim()

capital_regions = pd.read_csv('../data/capital_regions.csv')\
                    .set_index('Country')\
                    .CapitalRegion\
                    .to_dict()
def get_capital_id(country) -> int:
    country_name = country.tag('name:en')
    area_id = nominatim.query(f"{capital_regions[country_name]}, {country_name}").areaId()
    return area_id

pharmacy_mapping = pd.read_csv('../data/pharmacy.csv')\
                     .set_index('ISO2')\
                     .Translation\
                     .to_dict()
pharmacy_mapping['en'] = None
def count_pharmacy(country) -> int:
    selector = ['"amenity"="pharmacy"']
    local_name = pharmacy_mapping.get(country.tag('default_language'), None)

    if local_name:
        selector += [f'"amenity"="{local_name}"']
        
    query = overpassQueryBuilder(area=get_capital_id(country),
                                 elementType=['node', 'way'],
                                 selector=selector,
                                 out='count')
    return overpass.query(query, timeout=60).countElements()

dimensions_pharmacy = OrderedDict({
    'country': OrderedDict({
        country.tag('name:en'): country
        for country in countries
    })
})

data_pharmacy = Data(count_pharmacy, dimensions_pharmacy)

[overpass] downloading data: [timeout:60][out:json];area(3600421151)->.searchArea;(node["amenity"="pharmacy"]["amenity"="farmácia"](area.searchArea);way["amenity"="pharmacy"]["amenity"="farmácia"](area.searchArea);); out count;
[nominatim] downloading data: search
[overpass] downloading data: [timeout:60][out:json];area(3604773565)->.searchArea;(node["amenity"="pharmacy"](area.searchArea);way["amenity"="pharmacy"](area.searchArea);); out count;
[nominatim] downloading data: search
[overpass] downloading data: [timeout:60][out:json];area(3604001311)->.searchArea;(node["amenity"="pharmacy"](area.searchArea);way["amenity"="pharmacy"](area.searchArea);); out count;
[overpass] downloading data: [timeout:60][out:json];area(3607322726)->.searchArea;(node["amenity"="pharmacy"](area.searchArea);way["amenity"="pharmacy"](area.searchArea);); out count;
[overpass] downloading data: [timeout:60][out:json];area(3601707699)->.searchArea;(node["amenity"="pharmacy"](area.searchArea);way["amenity"="phar

In [101]:
df_pharmacy = data_pharmacy.getDataFrame().sort_values('value', ascending=False)
df_pharmacy.head()

,value
country,
Bangladesh,1187
Uzbekistan,865
Taiwan,285
Azerbaijan,185
Ethiopia,164


It's really strange, but country with most number of pharmacies in capital region is Bangladesh. 

In [103]:
country_2 = df_pharmacy.iloc[0].name
country_2

'Bangladesh'

## Задание №3. Поездка по Нью-Йорку (маршрут - 20 баллов, визуализация - 10 баллов).

Добраться __на автомобиле__ от входа в ``Central Park`` __Нью-Йорка__ (со стороны ``5th Avenue``) до пересечения ``Water Street`` и ``Washington Street`` в Бруклине (откуда получаются лучшие фото Манхэттенского моста) довольно непросто - разумеется, из-за вечных пробок. Однако еще сложнее это сделать, проезжая мимо школ, где дети то и дело переходят дорогу в неположенном месте.

Вам необходимо построить описанный выше маршрут, избегая на своем пути школы. Визуализируйте данный маршрут (также добавив школы и недоступные для проезда участки дорог) при помощи ``Folium``

Данные о расположении школ Нью-Йорка можно найти [здесь](https://catalog.data.gov/dataset/2019-2020-school-point-locations)

In [ ]:
# put your code here